## Import

In [1]:
import h5py # .h5 파일을 읽기 위한 패키지
import random
import pandas as pd
import numpy as np
import os
import glob
import torchvision.transforms as transforms # 이미지 변환 툴
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

C:\Users\OH\Anaconda3\envs\pytorch1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [3]:
CFG = {
    'EPOCHS':1000,
    'LEARNING_RATE':0.02,
    'BATCH_SIZE':128,
    'SEED':41
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [5]:
all_df = pd.read_csv('./train.csv')
all_points = h5py.File('./train.h5', 'r')

In [6]:
train_df = all_df.iloc[:int(len(all_df)*0.8)]
val_df = all_df.iloc[int(len(all_df)*0.8):]

## CustomDataset

In [7]:
class CustomDataset(Dataset):
    def __init__(self, id_list, label_list, point_list, train_mode=True, transforms=None):
        self.train_mode = train_mode
        self.transforms = transforms
        self.id_list = id_list
        self.label_list = label_list
        self.point_list = point_list
        
    def __getitem__(self, index):
        image_id = self.id_list[index]
        
        # h5파일을 바로 접근하여 사용하면 학습 속도가 병목 현상으로 많이 느릴 수 있습니다.
        points = self.point_list[str(image_id)][:]
        image = self.get_vector(points)
        
        if self.label_list is not None:
            label = self.label_list[index]
            return torch.Tensor(image).unsqueeze(0), label
        else:
            return torch.Tensor(image).unsqueeze(0)
    
    def get_vector(self, points, x_y_z=[18, 18, 18]):
        # 3D Points -> [16,16,16]
        xyzmin = np.min(points, axis=0) - 0.001
        xyzmax = np.max(points, axis=0) + 0.001

        diff = max(xyzmax-xyzmin) - (xyzmax-xyzmin)
        xyzmin = xyzmin - diff / 2
        xyzmax = xyzmax + diff / 2

        segments = []
        shape = []

        for i in range(3):
            # note the +1 in num 
            if type(x_y_z[i]) is not int:
                raise TypeError("x_y_z[{}] must be int".format(i))
            s, step = np.linspace(xyzmin[i], xyzmax[i], num=(x_y_z[i] + 1), retstep=True)
            segments.append(s)
            shape.append(step)

        n_voxels = x_y_z[0] * x_y_z[1] * x_y_z[2]
        n_x = x_y_z[0]
        n_y = x_y_z[1]
        n_z = x_y_z[2]

        structure = np.zeros((len(points), 4), dtype=int)
        structure[:,0] = np.searchsorted(segments[0], points[:,0]) - 1
        structure[:,1] = np.searchsorted(segments[1], points[:,1]) - 1
        structure[:,2] = np.searchsorted(segments[2], points[:,2]) - 1

        # i = ((y * n_x) + x) + (z * (n_x * n_y))
        structure[:,3] = ((structure[:,1] * n_x) + structure[:,0]) + (structure[:,2] * (n_x * n_y)) 

        vector = np.zeros(n_voxels)
        count = np.bincount(structure[:,3])
        vector[:len(count)] = count

        vector = vector.reshape(n_z, n_y, n_x)
        return vector

    def __len__(self):
        return len(self.id_list)

In [8]:
# train_transform = transforms.Compose([
#                     transforms.ToPILImage(),
#                     transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]), #이미지 사이즈 변형
#                     transforms.RandomRotation(30),
#                     # transforms.RandomErasing(scale=(0.025, 0.025), ratio=(1, 1)),
#                     # transforms.RandomPerspective(),
# transforms.LinearTransformation(),
#                     # transforms.Resize((200, 300)),
#                     # transforms.RandomCrop((180, 240)),
#                     transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
#                     transforms.GaussianBlur(51, (0.1, 2.0)),
#                     transforms.RandomHorizontalFlip(),
#                     transforms.RandomVerticalFlip(),
# transforms.grayscale(),
#                     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
#                     ])

# test_transform = transforms.Compose([
#                     transforms.ToPILImage(),
#                     transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]), #이미지 사이즈 변형
#                     transforms.RandomRotation(30),
#                     # transforms.RandomErasing(scale=(0.025, 0.025), ratio=(1, 1)),
#                     # transforms.RandomPerspective(),
# transforms.LinearTransformation(),
#                     # transforms.Resize((200, 300)),
#                     # transforms.RandomCrop((180, 240)),
#                     transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
#                     transforms.GaussianBlur(51, (0.1, 2.0)),
#                     transforms.RandomHorizontalFlip(),
#                     transforms.RandomVerticalFlip(),
# transforms.grayscale(),
#                     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
#                     ])

In [9]:
# def get_mean_std(dataset):
#     meanRGB = [np.mean(image.numpy(),axis=(1,2)) for image, _ in dataset]
#     stdRGB = [np.std(image.numpy(),axis=(1,2)) for image,_ in dataset]
    
#     meanR = np.mean([m[0] for m in meanRGB])
#     meanG = np.mean([m[1] for m in meanRGB])
#     meanB = np.mean([m[2] for m in meanRGB])
    
#     stdR = np.mean([s[0] for s in stdRGB])
#     stdG = np.mean([s[1] for s in stdRGB])
#     stdB = np.mean([s[2] for s in stdRGB])
    
#     print(meanR, meanG, meanB)
#     print(stdR, stdG, stdB)

#     train_transform = transforms.Compose([
#                         # transforms.RandomResizedCrop(224),
#                         # transforms.Resize(256),
#                         # transforms.RandomRotation(60),
#                         transforms.RandomPerspective(),
#                         transforms.ToTensor(), #이미지 데이터를 tensor
#                         transforms.Normalize((meanR, meanG, meanB), (stdR, stdG, stdB)), #이미지 정규화
#                         transforms.RandomHorizontalFlip(),
#                         transforms.RandomVerticalFlip(),
#                         # transforms.RandomErasing()
#                         ])

#     test_transform = transforms.Compose([
#                         # transforms.CenterCrop(224),
#                         transforms.ToTensor(),
#                         transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
#                         # transforms.RandomHorizontalFlip(),
#                         # transforms.RandomVerticalFlip()
#                         ])

In [10]:
train_transform = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(), #이미지 데이터를 tensor
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)), #이미지 정규화
                    transforms.RandomHorizontalFlip(),
                    # transforms.RandomVerticalFlip(),
                    # transforms.RandomErasing()
                    ])

test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
                    ])

In [11]:
# train_dataset = CustomDataset(train_df['ID'].values, train_df['label'].values, all_points)
# train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# val_dataset = CustomDataset(val_df['ID'].values, val_df['label'].values, all_points)
# val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [12]:
train_dataset = CustomDataset(train_df['ID'].values, train_df['label'].values, all_points, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['ID'].values, val_df['label'].values, all_points, train_mode=True, transforms=test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [13]:
train_dataset[0][0].shape, train_dataset[0][1]

(torch.Size([1, 18, 18, 18]), 5)

## Model Define

In [14]:
# class BaseModel(nn.Module):
#     def __init__(self):
#         super(BaseModel,self).__init__()
#         self.feature_extract = nn.Sequential(
#             nn.Conv3d(1,8,3),
#             nn.ReLU(),
#             nn.BatchNorm3d(8),
#             nn.Conv3d(8,32,3),
#             nn.ReLU(),
#             nn.BatchNorm3d(32),
#             nn.MaxPool3d(4),
#             nn.Conv3d(32,32,3),
#             nn.ReLU(),
#         )
#         self.classifier = nn.Linear(32,10)

#     def forward(self,x):
#         x = self.feature_extract(x)
#         x = x.view(x.size()[0],-1)
#         x = self.classifier(x)
#         return x

In [15]:
class Vgg19(nn.Module):
    def __init__(self):
        super(Vgg19,self).__init__()
        self.feature_extract = nn.Sequential(
            
            # 1
            nn.Conv3d(1,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(64),
            
            nn.Conv3d(64,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(64),
            nn.MaxPool3d(kernel_size = 2, stride = 2),
            
            # 2
            nn.Conv3d(64,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(128),
            
            nn.Conv3d(128,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(128),
            nn.MaxPool3d(kernel_size = 2, stride = 2),
            
            # 3
            nn.Conv3d(128,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(256),
            
            nn.Conv3d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(256),
            
            nn.Conv3d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(256),
            
            nn.Conv3d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(256),
            nn.MaxPool3d(kernel_size = 2, stride = 2),
            
            # 4
            nn.Conv3d(256,512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(512),
            
            nn.Conv3d(512,512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(512),
            
            nn.Conv3d(512,512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(512),
            
            nn.Conv3d(512,512, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm3d(512),
            nn.MaxPool3d(kernel_size = 2, stride = 2),
            
            # 5
#             nn.Conv3d(512,512, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.BatchNorm3d(512),
            
#             nn.Conv3d(512,512, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.BatchNorm3d(512),
            
#             nn.Conv3d(512,512, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.BatchNorm3d(512),
            
#             nn.Conv3d(512,512, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.BatchNorm3d(512),
#             nn.MaxPool3d(kernel_size = 2, stride = 2),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(512,10)
        )
        
    def forward(self,x):
        x = self.feature_extract(x)
        x = x.view(x.size()[0],-1)
        x = self.classifier(x)
        return x

In [48]:
# 고차원, 복잡하게 말고 간단하게, data augmentation,

In [49]:
# from torchsummary import summary
# model = BaseModel().to(device)
# summary(model, (1, 20, 20, 20))

## Train

In [50]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    best_loss = np.inf
    cnt = 0
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for data, label in tqdm(iter(train_loader)):
            data, label = data.float().to(device), label.long().to(device)
            optimizer.zero_grad()
            
            output = model(data)
            loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        if scheduler is not None:
            scheduler.step()
            
        val_loss, val_acc = validation(model, criterion, val_loader, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss)}] Val Loss : [{val_loss}] Val ACC : [{val_acc}]')
        
        if best_loss >= val_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), './best_model.pth')
            print('Model Saved.')
            cnt = 0
            
        if best_loss <= val_loss:
            cnt = cnt + 1 
            print('cnt: ' , cnt)
            if cnt > 5:
                break

In [51]:
def validation(model, criterion, val_loader, device):
    model.eval()
    true_labels = []
    model_preds = []
    val_loss = []
    with torch.no_grad():
        for data, label in tqdm(iter(val_loader)):
            data, label = data.float().to(device), label.long().to(device)
            
            model_pred = model(data)
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
    
    return np.mean(val_loss), accuracy_score(true_labels, model_preds)

In [52]:
from torchsummary import summary
model = Vgg19().to(device)
summary(model, (1, 18, 18, 18))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 18, 18, 18]           1,792
              ReLU-2       [-1, 64, 18, 18, 18]               0
       BatchNorm3d-3       [-1, 64, 18, 18, 18]             128
            Conv3d-4       [-1, 64, 18, 18, 18]         110,656
              ReLU-5       [-1, 64, 18, 18, 18]               0
       BatchNorm3d-6       [-1, 64, 18, 18, 18]             128
         MaxPool3d-7          [-1, 64, 9, 9, 9]               0
            Conv3d-8         [-1, 128, 9, 9, 9]         221,312
              ReLU-9         [-1, 128, 9, 9, 9]               0
      BatchNorm3d-10         [-1, 128, 9, 9, 9]             256
           Conv3d-11         [-1, 128, 9, 9, 9]         442,496
             ReLU-12         [-1, 128, 9, 9, 9]               0
      BatchNorm3d-13         [-1, 128, 9, 9, 9]             256
        MaxPool3d-14         [-1, 128, 

In [53]:
# pip install torchsummary

## Run!!

In [54]:
model = Vgg19()
model.eval()
# optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"], momentum=0.9)
# scheduler = None
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0.001, last_epoch=-1, verbose=False)

train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:33<00:00,  2.38it/s]


Epoch : [1] Train Loss : [1.3371462882898104] Val Loss : [0.4535674579158614] Val ACC : [0.8618]
Model Saved.
cnt:  1


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:32<00:00,  2.43it/s]


Epoch : [2] Train Loss : [0.3044026960342075] Val Loss : [0.293890877992292] Val ACC : [0.9091]
Model Saved.
cnt:  1


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:33<00:00,  2.39it/s]


...

Epoch : [31] Train Loss : [0.007473624717289504] Val Loss : [0.06098742735775886] Val ACC : [0.9871]
cnt:  4


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:31<00:00,  2.48it/s]


Epoch : [32] Train Loss : [0.007840291055581823] Val Loss : [0.07006788695536117] Val ACC : [0.9849]
cnt:  5


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:31<00:00,  2.48it/s]

Epoch : [33] Train Loss : [0.006463232028919721] Val Loss : [0.06525040377988561] Val ACC : [0.9861]
cnt:  6


# 학습 테러 하고 도망치기 키키키키키 

## Inference

In [55]:
test_df = pd.read_csv('./sample_submission.csv')
test_points = h5py.File('./test.h5', 'r')

In [56]:
test_dataset = CustomDataset(test_df['ID'].values, None, test_points)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [57]:
checkpoint = torch.load('./best_model.pth')
model = Vgg19()
model.load_state_dict(checkpoint)
model.eval()

Vgg19(
  (feature_extract): Sequential(
    (0): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU()
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (4): ReLU()
    (5): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (8): ReLU()
    (9): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (11): ReLU()
    (12): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv3d(128, 256, k

In [58]:
def predict(model, test_loader, device):
    model.to(device)
    model.eval()
    model_preds = []
    with torch.no_grad():
        for data in tqdm(iter(test_loader)):
            data = data.float().to(device)
            
            batch_pred = model(data)
            
            model_preds += batch_pred.argmax(1).detach().cpu().numpy().tolist()
    
    return model_preds

In [59]:
preds = predict(model, test_loader, device)

100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [02:39<00:00,  1.96it/s]


## Submit

In [60]:
test_df['label'] = preds
test_df.to_csv('./submit28.csv', index=False)

In [61]:
# from keras.datasets import mnist
# import matplotlib.pyplot as plt

# plt.plot(model.train_loss['train_loss'])
# plt.plot(model.val_loss['val_loss'])
# plt.title('model loss')
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.legend(['train','val'], loc = 'upper left')
# plt.show()

In [62]:
# def get_mean_std(dataset):
#     meanRGB = [np.mean(image.numpy(),axis=(1,2)) for image, _ in dataset]
#     stdRGB = [np.std(image.numpy(),axis=(1,2)) for image,_ in dataset]
    
#     meanR = np.mean([m[0] for m in meanRGB])
#     meanG = np.mean([m[1] for m in meanRGB])
#     meanB = np.mean([m[2] for m in meanRGB])
    
#     stdR = np.mean([s[0] for s in stdRGB])
#     stdG = np.mean([s[1] for s in stdRGB])
#     stdB = np.mean([s[2] for s in stdRGB])
    
#     print(meanR, meanG, meanB)
#     print(stdR, stdG, stdB)


In [63]:
# print(meanR, meanG, meanB)
# print(stdR, stdG, stdB)

In [64]:
# import numpy as np

# def get_mean_std(dataset):
#     meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in dataset]
#     stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in dataset]

#     meanR = np.mean([m[0] for m in meanRGB])
#     meanG = np.mean([m[1] for m in meanRGB])
#     meanB = np.mean([m[2] for m in meanRGB])

#     stdR = np.mean([s[0] for s in stdRGB])
#     stdG = np.mean([s[1] for s in stdRGB])
#     stdB = np.mean([s[2] for s in stdRGB])

# print(meanR, meanG, meanB)
# print(stdR, stdG, stdB)